In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [4]:
#df = pd.read_json('./train.json/train.json',lines=True)
df = pd.read_json('./test.json/test.json',lines=True)

In [7]:
df['summary'].isnull().sum()

0

In [6]:
# df = df[df['summary'].notna()]
df['summary'].fillna(df['reviewText'], inplace=True)

In [12]:
#batch_1 = df[['summary','overall']]
batch_1 = df['summary']

0       Fantastic mix of "old school" with a creative ...
1                 Digitally Extracted Stereo (DES) Rules!
2                               Excellent unplugged album
3                                     another masterpiece
4                                       True Classic Rock
                              ...                        
9995            3 1/2 stars for a really good compilation
9996                                          Three Stars
9997                             Glad to have another gem
9998                                          Three Stars
9999          Kelly did a very good job in making that CD
Name: summary, Length: 10000, dtype: object

In [9]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

In [10]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [14]:
#tokenized = batch_1['summary'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized = batch_1.apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 77,pad_to_max_length = True)))

In [15]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [16]:
np.array(padded).shape

(10000, 77)

In [17]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10000, 77)

In [18]:
input_ids = torch.LongTensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    for i in range(10):
        last_hidden_states = model(input_ids[i*1000:(i+1)*1000], attention_mask=attention_mask[i*1000:(i+1)*1000])
        if i == 0:
            features = last_hidden_states[0][:,0,:].numpy()
        else:
            temp =  last_hidden_states[0][:,0,:].numpy()
            features = np.concatenate((features, temp), axis=0)
        print(i)
        print(features.shape)
        

0
(1000, 768)
1
(2000, 768)
2
(3000, 768)
3
(4000, 768)
4
(5000, 768)
5
(6000, 768)
6
(7000, 768)
7
(8000, 768)
8
(9000, 768)
9
(10000, 768)


In [20]:
pd.DataFrame(features).to_csv('features_test.csv')